In [1]:
import requests
from bs4 import BeautifulSoup
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}

In [2]:
def getDownload(url, parmas = None, retries = 3):
    resp = None
    
    try:
        resp = requests.get(url, params = param, headers = header)
        resp.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retries > 0:
            print('Retries: {0}'.format(retries))
            return getDownload(url, param, retries - 1)
        else:
            print(resp.status_code)
            print(resp.reason)
            print(resp.request.headers)
            
    return resp

In [3]:
url = 'http://example.webscraping.com/places/ajax/search.json'
param = {
    'search_term':'korea',
    'page_size':10,
    'page':0
} 
html = getDownload(url,param)
html.headers

{'Server': 'nginx', 'Date': 'Fri, 10 May 2019 07:16:21 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'X-Powered-By': 'web2py', 'Set-Cookie': 'session_id_places=True; httponly; Path=/', 'Expires': 'Fri, 10 May 2019 07:16:21 GMT', 'Pragma': 'no-cache', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Content-Encoding': 'gzip'}

In [4]:
for _ in html.json()['records']:
    print(_['pretty_link'])
    dom = BeautifulSoup(_['pretty_link'], 'lxml')
    print([_['href'] for _ in dom.find_all('a')])
    
    break

<div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png" /> North Korea</a></div>
['/places/default/view/North-Korea-165']


In [49]:
import re


text = html.json()['records'][0]['pretty_link']
group = re.findall("<a href=\"(.+)\"><img src=\"(.+)\" /> (.+)</a>",text)
requests.compat.urljoin(url, group[0][0]), \
requests.compat.urljoin(url, group[0][1], group[0][2])

('http://example.webscraping.com/places/default/view/North-Korea-165',
 'http://example.webscraping.com/places/static/images/flags/kp.png')

# Selenium

In [50]:
from selenium import webdriver

In [173]:
browser = webdriver.Chrome()

In [53]:
url = 'http://example.webscraping.com/places/default/search'
browser.get(url)

## id

In [57]:
browser.find_element_by_id('search_term').clear()
browser.find_element_by_id('search_term').send_keys('korea')
browser.find_element_by_id('search').click()

## css_selector

+ Selector => Tag, ID  <br>
(#아이디), CLASS(.클래스명)
+ [name = 'asdf'] <br>
+ [name ^= 'asdfasd'] <br>
+ [name $= 'asdfasd'] <br>
+ div p => find_all(div 찾고, 자손중에 p) <br>
+ div > p => find_all(recursive = False) <br>
+ div 찾고, 자식 p <br>
+ div + p => find(div).find_next_sibling(P) <br>

In [59]:
browser.find_element_by_css_selector("#search_term")
browser.find_element_by_css_selector("#search_term").send_keys('korea')
browser.find_element_by_css_selector("#search_term").click()

## x_path

+ xpath => xml tree 탐색 <br>
+ . => 현재위치 <br>
+ // => 자손 <br>
+ / => 자식 <br>
+ [@어트리뷰트 = "값"] <br>
+ id = results <br>
+ [@id = 'results'] 

In [103]:
browser.find_element_by_xpath('//input[@id="search_term"]').clear()
browser.find_element_by_xpath('//input[@id="search_term"]').send_keys('korea')
browser.find_element_by_xpath('//input[@id="search"]').click()

In [106]:
for tag in browser.find_elements_by_xpath('//div[@id="results"]//a'):
    print(tag.text)
    print(tag.get_attribute('href'))

North Korea
http://example.webscraping.com/places/default/view/North-Korea-165
South Korea
http://example.webscraping.com/places/default/view/South-Korea-211


In [107]:
%%writefile account.json
{
    "id":"achrisfy7",
    "pw":"5983176y"
}

Overwriting account.json


In [109]:
import json

In [110]:
with open('account.json') as f:
    account = json.load(f)

In [111]:
browser.get('https://nid.naver.com/nidlogin.login')


## id

In [112]:
browser.find_element_by_id('id').clear()
browser.find_element_by_id('pw').clear()
browser.find_element_by_id('id').send_keys(account['id'])
browser.find_element_by_id('pw').send_keys(account['pw'])

## css

In [113]:
browser.find_element_by_css_selector('#id').clear()
browser.find_element_by_css_selector('#pw').clear()
browser.find_element_by_css_selector('#id').send_keys(account['id'])
browser.find_element_by_css_selector('#pw').send_keys(account['pw'])

## xpath

In [117]:
browser.find_element_by_xpath('//input[@name="id"]').clear()
browser.find_element_by_xpath('//input[@name="pw"]').clear()
browser.find_element_by_xpath('//input[@name="id"]').send_keys(account["id"])
browser.find_element_by_xpath('//input[@name="pw"]').send_keys(account["pw"])

In [118]:
browser.find_element_by_class_name('btn_global').click()


# 받은메일함

In [119]:
browser.get('https://mail.naver.com/')

In [123]:
for tag in browser.find_elements_by_css_selector('strong.mail_title'):
    print(tag.text)

(광고) [스타벅스] SSG카드로 스타벅스를 이용하시고, e-Gift 카드 3만원권 혜택을 누리세요!
(광고) MIXXO 147종 균일가 14,900원 찬스 +무배
(광고)[L.POINT] 5월 포인트 명세서 양세원님의 포인트는?
(광고) [티몬 9번째 생일파티] 단 하루 특별한 혜택을 놓치지 마세요!
(광고) 에어팟 2세대 화이트 한정수량 99,000원 +무료배송
(광고) 딱 하루! 배송비 없는 디지털데이 특가 시작 +쿠폰 이벤트
(광고) 동원 소불고기 3팩 외 균일가 9,900원 +무료배송
새로운 환경에서 로그인 되었습니다.
05월09일 양세원님께 꼭 맞는 채용정보가 도착했어요.
2019년 상반기 중앙대학교 캠퍼스리크루팅 일정(Ver.20190510)
(광고) 양세원님, 배송비 없는 해외직구 시작합니다!
(광고) 가성비 갑! 오뚜기 핫도그 20개 11,900원 +무배+무제한적립
(광고) 데일리룩 필수템! 남녀공용 무지 반팔티 1,900원 +무배
05월08일 양세원님께 꼭 맞는 채용정보가 도착했어요.
(광고) 오늘 단 하루 전 상품 무료배송! +깜짝 쿠폰 이벤트


# 스팸메일함

In [124]:
browser.find_element_by_css_selector('span.item_wrap.bu6').click()

In [130]:
for tag in browser.find_elements_by_css_selector("strong.mail_title"):
    print(tag.text)

[무료창업설명회]창업아이템으로 이런 건 어떠세요?


## Cookies

In [131]:
browser.get_cookies()

[{'domain': '.naver.com',
  'httpOnly': False,
  'name': 'NID_JKL',
  'path': '/',
  'secure': True,
  'value': 'SP4+nxqEtnnrColxRwRp2/UsX3xwmFQxwBgB2Y5HKKs='},
 {'domain': '.naver.com',
  'expiry': 2524640375.472232,
  'httpOnly': False,
  'name': 'NNB',
  'path': '/',
  'secure': False,
  'value': 'PXWDQJYKGTKVY'},
 {'domain': 'mail.naver.com',
  'httpOnly': False,
  'name': 'NMUSER',
  'path': '/',
  'secure': False,
  'value': 'uQblKqEwaqbsaAUqKxKlM6Fcb40qpNvXaqu9FxbdFqMrFovsFxuZKAnmaqns3xnDKxEs6xRaaZnladUstoRVadUsadUs3ons1rejL9Us6xRVaqnD16lvpB2RKSl5WLl5MBp0bSloWrdnaAvmKARGW430DVdZ7BF0Mre5pzJZDLl5WLl5MBp0bSloWrdnaAvmKAn='},
 {'domain': '.naver.com',
  'httpOnly': False,
  'name': 'nid_inf',
  'path': '/',
  'secure': False,
  'value': '1635658415'},
 {'domain': '.naver.com',
  'httpOnly': True,
  'name': 'NID_AUT',
  'path': '/',
  'secure': False,
  'value': '4kpshTpZ0grVJZDspP9o/IHt4I7r/hAbCBAVor/z0oCcXhsu4q3IkdA9ubw0IaoR'},
 {'domain': '.naver.com',
  'httpOnly': False,
  'name'

In [134]:
browser.get('https://mail.naver.com')
# browser.page_source

In [135]:
html = getDownload('https://mail.naver.com')
# html.text

In [132]:
from requests import Session

session = Session()

In [138]:
for cookie in browser.get_cookies():
    print(cookie)
    print()

{'domain': '.naver.com', 'httpOnly': False, 'name': 'NID_JKL', 'path': '/', 'secure': True, 'value': 'SP4+nxqEtnnrColxRwRp2/UsX3xwmFQxwBgB2Y5HKKs='}

{'domain': '.naver.com', 'expiry': 2524640375.472232, 'httpOnly': False, 'name': 'NNB', 'path': '/', 'secure': False, 'value': 'PXWDQJYKGTKVY'}

{'domain': 'mail.naver.com', 'httpOnly': False, 'name': 'NMUSER', 'path': '/', 'secure': False, 'value': 'uQblKqEwaqbsaAUqKxKlM6Fcb40qpNvXaqu9FxbdFqMrFovsFxuZKAnmaqns3xnDKxEs6xRaaZnladUstoRVadUsadUs3ons1rejL9Us6xRVaqnD16lvpB2RKSl5WLl5MBp0bSloWrdnaAvmKARGW430DVdZ7BF0Mre5pzJZDLl5WLl5MBp0bSloWrdnaAvmKAn='}

{'domain': '.naver.com', 'httpOnly': False, 'name': 'nid_inf', 'path': '/', 'secure': False, 'value': '1635658415'}

{'domain': '.naver.com', 'httpOnly': True, 'name': 'NID_AUT', 'path': '/', 'secure': False, 'value': '4kpshTpZ0grVJZDspP9o/IHt4I7r/hAbCBAVor/z0oCcXhsu4q3IkdA9ubw0IaoR'}

{'domain': '.naver.com', 'httpOnly': False, 'name': 'NID_SES', 'path': '/', 'secure': False, 'value': 'AAABeTaiL

In [141]:
for cookie in browser.get_cookies():
    print(cookie['name'], cookie['value'])
    session.cookies.set(cookie['name'], cookie['value'])
    

NID_JKL SP4+nxqEtnnrColxRwRp2/UsX3xwmFQxwBgB2Y5HKKs=
NNB PXWDQJYKGTKVY
NMUSER uQblKqEwaqbsaAUqKxKlM6Fcb40qpNvXaqu9FxbdFqMrFovsFxuZKAnmaqns3xnDKxEs6xRaaZnladUstoRVadUsadUs3ons1rejL9Us6xRVaqnD16lvpB2RKSl5WLl5MBp0bSloWrdnaAvmKARGW430DVdZ7BF0Mre5pzJZDLl5WLl5MBp0bSloWrdnaAvmKAn=
nid_inf 1635658415
NID_AUT 4kpshTpZ0grVJZDspP9o/IHt4I7r/hAbCBAVor/z0oCcXhsu4q3IkdA9ubw0IaoR
NID_SES AAABeTaiLCDuvDW0EVQqDUYOoDmfL7mzkVdrxskdDrv4msk/9mlbr7r1WG/wABhupqC7PhsWyeraThVaDH0Vqv9heXJWfgDesESoDKvfD+9mZedvt1eEbejz9AY1duGDGe/m654u3zTaXl6yrGi+8cyYKyM1f0N48iL3HstJab0SvniBIHonJPVqqha9PLOyDTAZDFMTGJRoPm4dTYNhvmuC5v46l6LkoBGnEVegXlOnNUEFnAMdWUGh1pGCXE+rNWVv+FDckH2ScyeiwWATlOATpcwvFc9GHy16O+UoQUu+E8l+DFyJ0PlT/LaFplP/sCZlPB8qmMlNdtESnJ0sR/nnvymiPKlpnlYVtYvyUz/D3DQS4yNUkMvJiwzWZqQpmBU9Hh3r9Zc7NOKVc0yDAVZSOYbCRrpfl2XLCHhT98cz9oD3szy/Ex656N0XLt6gZGq8Ql+tiQiHZFAUjYnMSbElRoA4qkvg6ckFGHzYfxx533rcRWdDmRL8Aa7exNePVgt8eA==


# IFrame

In [ ]:
# html = session.get('https://mail.naver.com')
# html.text

In [142]:
browser.get('https://www.daum.net/')

In [151]:
iframe = browser.find_element_by_id('loginForm')
browser.switch_to.frame(iframe)
# browser.find_element_by_css_selector('.lab_login.lab_id').send_keys()

In [152]:
browser.find_element_by_id('id').get_attribute('outerHTML')
browser.find_element_by_id('id').send_keys('achrisfy7')
browser.find_element_by_id('inputPwd').send_keys('5983176t')


In [153]:
browser.find_element_by_id('loginSubmit').click()

# 새창

In [154]:
url = 'https://www.kt.com/'
browser.get(url)

In [155]:
browser.find_element_by_css_selector('.quick > a:first-child').get_attribute('outerHTML')


'<a href="javascript:floating.login();" title="새창 열림">로그인</a>'

In [160]:
browser.find_element_by_css_selector('.quick > a:first-child').click()


In [161]:
# browser.find_element_by_id('userId') => 새창에서 작업하고 있지 않기 때문에 error
browser.window_handles

['CDwindow-40308B8A5BD36858ED8D6194CFC81AE6',
 'CDwindow-CA8BB63BF73E4BA70F709D3E4D01005A']

In [162]:
browser.switch_to.window(browser.window_handles[1])

In [164]:
browser.find_element_by_id('userId').send_keys('test')


![12](https://user-images.githubusercontent.com/40786348/57563055-2b67e180-73d4-11e9-83f7-13e7bff882e1.PNG)


In [171]:
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait

wait = webdriver.support.ui.WebDriverWait(browser, 10, 0.5, NoSuchElementException)

In [ ]:
url = 'http://example.webscraping.com/places/default/search'
browser.get(url)

browser.find_element_by_id('search_term').clear()
browser.find_element_by_xpath('//input[@id="search_term"]').send_keys('korea')
browser.find_element_by_id('search').click()

wait.until(lambda x:x.find_element_by_xpath('//div[@id="results"]//a'))

for _ in browser.find_elements_by_xpath('//div[@id="results"]//a'):
    print(_.tag_name)
    print(_.text)
    print(_.get_attribute('href'))